
# Annotate documents for named entity recognition

Select the document to tag and the output folder to save the annotations to.


In [1]:
from src.notebook.file_chooser import choose
choose()

Document to tag (.txt file):


FileChooser(path='/home/abertsch/projects/ADEPTLab/annotate', filename='', title='HTML(value='', layout=Layout…

Output folder to save annotations to:


FileChooser(path='/home/abertsch/projects/ADEPTLab/annotate', filename='', title='HTML(value='', layout=Layout…

Now run the cell below to annotate the document! You can type a number (+enter) to label an entity, or just press enter to skip any words that are not entities.

In [6]:
from src.annotate import setup_doc, takedown, get_tag, not_done
from IPython.display import clear_output
    
setup_doc()

while(not_done()):    
    clear_output(wait=True)
    get_tag()
    
takedown()

Progress: [===       ] - 34% of 95 tokens
TAG OPTIONS: (press enter to leave untagged, b to go back, p to save progress and exit)
0 people, including fictional		4 companies, institutions, etc.
1 nationalities, religions		5 countries, cities, states
2 mountains, rivers, etc.		6 events--named hurricanes, etc
3 buildings, airports, etc.		7 measurements (e.g. weight, distance)
world, according to Swiss National  <<Tourism>>  Department spokesperson Jan Smith. The
	TAG? p
THIS IS WHAT IS DUMPED:
The: position 0, length 3, tag -1
Alps: position 4, length 4, tag 2
are: position 9, length 3, tag -1
the: position 13, length 3, tag -1
most: position 17, length 4, tag -1
beautiful: position 22, length 9, tag -1
mountains: position 32, length 9, tag -1
in: position 42, length 2, tag -1
the: position 45, length 3, tag -1
world,: position 49, length 6, tag -1
according: position 56, length 9, tag -1
to: position 66, length 2, tag -1
Swiss: position 66, length 5, tag 4
National: position 72, length 8